<a href="https://colab.research.google.com/github/htapiagroup/sistdist/blob/master/MCA_SDU_S2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistemas Distribuidos y Ubicuos II 

## MCA Tercer cuatrimestre

### Horacio Tapia-McClung



## Resumen Semana 1

* Como "guarda" informacion la computadora

In [2]:
import numpy as np
import pandas as pd
import psutil, os

def huella_memoria():
  '''Regresa la huella de memoria en MB usada por un proceso de Python'''
  mem = psutil.Process(os.getpid()).memory_info().rss
  return (mem / 1024**2)


antes = huella_memoria()
N = (1024**2)//8
x = np.random.rand(50*N)
despues = huella_memoria()
print('Memoria asignada a un arreglo: {} MB'.format(despues-antes))
print('Memoria utilizada por el arreglo: {} MB'.format(x.nbytes / (1024**2)))

antes = huella_memoria()
x**2
despues = huella_memoria()
print('Memoria asignada a una operacion: {} MB'.format(despues-antes))

df = pd.DataFrame(x)
print('Memoria utilizada por el DataFrame: {} MB'.format(df.memory_usage(index=False)/(1024**2)))



Memoria asignada a un arreglo: 0.0 MB
Memoria utilizada por el arreglo: 50.0 MB
Memoria asignada a una operacion: 49.96484375 MB
Memoria utilizada por el DataFrame: 0    50.0
dtype: float64 MB


# Semana 2

Ahora veremos como acceder a un conjunto de datos separandolo en partes y como podemos conocer las propiedades del conjunto compleo conociendo las propiedades de cada parte y agregando al final.
Esto es util y practico cuando tenemos un conjunto de datos grande como el siguiente:

In [5]:
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-01.csv

--2019-05-13 17:24:22--  https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-01.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.178.93
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.178.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 772098307 (736M) [text/csv]
Saving to: ‘yellow_tripdata_2018-01.csv’

yellow_tripdata_201 100%[===================>] 736.33M  43.7MB/s    in 17s     

2019-05-13 17:24:39 (42.7 MB/s) - ‘yellow_tripdata_2018-01.csv’ saved [772098307/772098307]



In [0]:
archivo = 'yellow_tripdata_2018-01.csv'

In [10]:
for trozo in pd.read_csv(archivo, chunksize=500000):
  print('type: %s shape %s' % (type(trozo), trozo.shape))

type: <class 'pandas.core.frame.DataFrame'> forma (500000, 17)
type: <class 'pandas.core.frame.DataFrame'> forma (500000, 17)
type: <class 'pandas.core.frame.DataFrame'> forma (500000, 17)
type: <class 'pandas.core.frame.DataFrame'> forma (500000, 17)
type: <class 'pandas.core.frame.DataFrame'> forma (500000, 17)
type: <class 'pandas.core.frame.DataFrame'> forma (500000, 17)
type: <class 'pandas.core.frame.DataFrame'> forma (500000, 17)
type: <class 'pandas.core.frame.DataFrame'> forma (500000, 17)
type: <class 'pandas.core.frame.DataFrame'> forma (500000, 17)
type: <class 'pandas.core.frame.DataFrame'> forma (500000, 17)
type: <class 'pandas.core.frame.DataFrame'> forma (500000, 17)
type: <class 'pandas.core.frame.DataFrame'> forma (500000, 17)
type: <class 'pandas.core.frame.DataFrame'> forma (500000, 17)
type: <class 'pandas.core.frame.DataFrame'> forma (500000, 17)
type: <class 'pandas.core.frame.DataFrame'> forma (500000, 17)
type: <class 'pandas.core.frame.DataFrame'> forma (5000

In [11]:
trozo.shape

(259874, 17)

Cada trozo tiene los atributos estandares de un `DataFrame`

In [12]:
trozo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259874 entries, 8500000 to 8759873
Data columns (total 17 columns):
VendorID                 259874 non-null int64
tpep_pickup_datetime     259874 non-null object
tpep_dropoff_datetime    259874 non-null object
passenger_count          259874 non-null int64
trip_distance            259874 non-null float64
RatecodeID               259874 non-null int64
store_and_fwd_flag       259874 non-null object
PULocationID             259874 non-null int64
DOLocationID             259874 non-null int64
payment_type             259874 non-null int64
fare_amount              259874 non-null float64
extra                    259874 non-null float64
mta_tax                  259874 non-null float64
tip_amount               259874 non-null float64
tolls_amount             259874 non-null float64
improvement_surcharge    259874 non-null float64
total_amount             259874 non-null float64
dtypes: float64(8), int64(6), object(3)
memory usage: 33.7+ MB


Veamos como filtrar un trozo usando una condicion, en este caso decidiremos que un viaje es largo si distancia del viaje es mayor a 10 millas ( [ver aqui](https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf).

In [0]:
es_un_viaje_largo = (trozo.trip_distance > 10)

In [16]:
trozo.loc[es_un_viaje_largo].shape

(11817, 17)

Ahora vamos a encapsular este filtrado en una funcion que acepta una estructura y regresa la estructura filtrada y luego vamos a iterar sobre los trozos

In [0]:
def filtra_es_viaje_largo(data):
  '''Returns DataFrame filtering trips with distance over 10 miles'''
  es_un_viaje_largo = (data.trip_distance > 10)
  return data.loc[es_un_viaje_largo]

In [0]:
trozos = []
for trozo in pd.read_csv(archivo, chunksize=500000):
  trozos.append